In [ ]:
import csv
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
from datetime import datetime
import dateutil.parser as parser 

In [ ]:
host_ip = "118.139.86.28"

In [ ]:
#reading the csv file
with open ("climate_streaming.csv", "r") as file:
    climate_file = file.readlines()
    climate_file = [line.strip() for line in climate_file]
    for i in range(len(climate_file)):
        climate_file[i] = climate_file[i].split(",")

In [ ]:
#adds a day to the date inputted
def addOneDay(currentDay):
    currDate = currentDay
    
    currDate = datetime.strptime(currDate, '%Y-%m-%dT%H:%M:%S')
    newDate = currDate + dt.timedelta(days = 1)
    newDate = newDate.strftime('%Y/%-m/%-d')
    newDate = parser.parse(newDate)
    
    return newDate.isoformat()


In [ ]:
#Set up KafkaProducer
def connect_kafka_producer():
    _producer = None
    try: 
        _producer = KafkaProducer(bootstrap_servers=[f'{host_ip}:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10),
                                 client_id = "climate")
        
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
        
    finally:
        return _producer

In [ ]:
#Function for publishing message
def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        
        print("Message successfully publish. Data: " + str(data))
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [ ]:
topic = "climateTopic"

print("Publishing records...")

producer = connect_kafka_producer()
prod_id = producer.config['client_id']

# latest date in climateLst:
newDate = '2023-01-01T00:00:00'

#While loop will keep publishing the data 
while True:

    randIndex = random.randint(1,len(climate_file)-1)

    newDate = addOneDay(newDate)
    row = climate_file[randIndex]

    climate_record = {
        "producer_info": prod_id,
        "station_id": int("220468"),
        "date": newDate,
        "latitude": float(row[0]),
        "longitude": float(row[1]),
        "air_temperature": int(row[2]),
        "relative_humidity": float(row[3]),
        "windspeed_knots": float(row[4]),
        "max_wind_speed": float(row[5]),
        "precipitation": row[6],
        "prec_val": float(row[6][1:5]),
        "prec_type": row[6][5],
        "GHI": int(row[7])
    }

    publish_message(producer, topic, 'climatedata', climate_record)
    sleep(10) #delay it to produce every 10 seconds 
    
